In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import SVC
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

# load in March Madness data, target is for regression, target2 is for classification
df = pd.read_csv('MarchMadnessData2024.csv')
target = df['margin']
target2 = (df['margin'] > 0)

# drop target column
X = df.drop(columns = ['margin'])

# shuffle the data set
shuffled_indices = np.random.permutation(X.index)
X = X.iloc[shuffled_indices].reset_index(drop=True)

print(X)

      Win-Loss Percentage_team1  SRS_team1  SOS_team1  Team Points_team1  \
0                         0.720      16.18       6.10             1917.0   
1                         0.375      -2.69       2.53             1749.0   
2                         0.840      13.82       4.65             2027.0   
3                         0.750      20.32       7.61             1993.0   
4                         0.750      20.32       7.61             1993.0   
...                         ...        ...        ...                ...   
2101                      0.538      -2.92      -2.33             1911.0   
2102                      0.154     -18.37      -2.24             1847.0   
2103                      0.560      -4.13      -2.67             2048.0   
2104                      0.750       5.49      -1.42             1804.0   
2105                      0.538      -1.31      -2.13             1987.0   

      Opponent Ponts_team1  Minutes Played_team1  FG_team1  FGA_team1  \
0             

In [3]:
max_score = 0
 # shuffle the data set
shuffled_indices = np.random.permutation(X.index)
X = X.iloc[shuffled_indices].reset_index(drop=True)

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

for components in range(14, 47): 
    # create a pca model for each number of components
    pca = PCA(n_components = components)
    X_pca = pca.fit_transform(X_norm)

    # calculate explained variance
    explained_variance = sum(pca.explained_variance_ratio_)
    
    # split data into train and test
    trainX, testX, trainY, testY = train_test_split(X_pca, target2, test_size = 0.2)
    scorerVar = make_scorer(f1_score, pos_label = 1)

    # create and fit svm model, print initial score
    svm_model = SVC()
    svm_model.fit(X_pca, target2)
    svm_model.predict(testX)
    print("Initial score for svm is ", svm_model.score(testX, testY), " for ", components, " components")
    
    # get the cross value score for svm, if it's better than the max score then set it to max
    svm_cv_score = cross_val_score(svm_model, X_pca, target2, cv = 10, scoring = scorerVar)
    print("Cross Validation Score for SVM: ", np.mean(svm_cv_score), " for ", components, " components")
    print()
    if np.mean(svm_cv_score) > max_score:
        max_score = np.mean(svm_cv_score)
        print("New max: ", max_score, " at ", components, "components")
    
    # repeat with logistic regression model
    lr_model = LogisticRegression()
    lr_model.fit(trainX, trainY)
    lr_model.predict(testX)
    print("Initial score for LogRegression is ", lr_model.score(testX, testY), " for ", components, " components")

    lr_cv_score = cross_val_score(lr_model, X_pca, target2, cv = 7, scoring = scorerVar)
    print("Cross Validation Score for LogRegression: ", np.mean(lr_cv_score), " for ", components, "components")
    print()
    if np.mean(lr_cv_score) > max_score:
        max_score = np.mean(lr_cv_score)
        print("New max: ", max_score, " at ", components, "components")
    
# # Best Score: Logistic Regression at 25 components: cv_score = 0.8410353628455827

Initial score for svm is  0.6943127962085308  for  14  components
Cross Validation Score for SVM:  0.8280465082502089  for  14  components

New max:  0.8280465082502089  at  14 components
Initial score for LogRegression is  0.6943127962085308  for  14  components
Cross Validation Score for LogRegression:  0.8280459342530558  for  14 components

Initial score for svm is  0.6777251184834123  for  15  components
Cross Validation Score for SVM:  0.8280465082502089  for  15  components

Initial score for LogRegression is  0.6777251184834123  for  15  components
Cross Validation Score for LogRegression:  0.8280459342530558  for  15 components

Initial score for svm is  0.7298578199052133  for  16  components
Cross Validation Score for SVM:  0.8280465082502089  for  16  components

Initial score for LogRegression is  0.7298578199052133  for  16  components
Cross Validation Score for LogRegression:  0.8280459342530558  for  16 components

Initial score for svm is  0.6706161137440758  for  17  

In [4]:
team1 = [
        'Connecticut', 'Fla Atlantic', 'San Diego St', 'Auburn', 'BYU', 'Illinois', 'Wash State', 'Iowa St', 
        'North Carolina', 'North Carolina', 'Miss State', 'St Marys', 'Alabama', 'Clemson', 'Baylor', 'Dayton', 'Arizona', 
        'Houston', 'Nebraska', 'Wisconsin', 'Duke', 'Texas Tech', 'Kentucky', 'Florida', 'Florida', 'Marquette', 
        'Purdue', 'Purdue', 'Utah St', 'Gonzaga', 'Kansas', 'S Carolina', 'Creighton', 'Texas', 'Texas', 'Tennessee'
        ]

team2 = [
        'Stetson', 'Northwestern', 'UAB', 'Yale', 'Duquesne', 'Morehead St', 'Drake', 'S Dakota St', 
        'Wagner', 'Howard', 'Michigan St', 'Grd Canyon', 'Col Charlestn', 'New Mexico', 'Colgate', 'Nevada', 'Lg Beach St', 
        'Longwood', 'Texas A&M', 'James Mad', 'Vermont', 'NC State', 'Oakland', 'Boise St', 'Colorado', 'W Kentucky', 
        'Montana St', 'Grambling St', 'TX Christian', 'McNeese St', 'Samford', 'Oregon', 'Akron', 'Virginia', 'Colorado St', 'St Peters'
        ]

location = ['neutral','home','away','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral',
            'neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral'
             'neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral','neutral'
        ]
# fill in teams and locations
print(len(team1))
print(len(team2))
print(len(location))

36
36
36


In [7]:
data_dump = pd.read_csv('march_madness_data_dump.csv')

df = pd.DataFrame({'team1':team1, 'team2':team2, 'team1_location':location})

# merge data dump with game data to predict
merge1 = pd.merge(df, data_dump, left_on = 'team1', right_on = 'School', how = 'left')
final = pd.merge(merge1, data_dump, left_on = 'team2', right_on = 'School', how = 'left', suffixes = ('_team1', '_team2'))

# use get_dummies() on team1_location, so that classification can be done 
final = pd.get_dummies(final, columns = ['team1_location'])

# remove all rows containing null or empty values
final = final.drop(columns = ['team1', 'team2', 'Rk_team1', 'Rk_team2', 'School_team1', 'School_team2'])

# MinMaxScalar was found to be the best normalizer option
scaler = MinMaxScaler()
final_norm = scaler.fit_transform(final)
X_norm = scaler.fit_transform(X)

# pca with 25 components was the best option
pca = PCA(n_components = 25)
final_pca = pca.fit_transform(final_norm)
X_pca = pca.fit_transform(X_norm)

X_train, X_test, y_train, y_test = train_test_split(X_pca, target2, test_size=0.25)

X_train = scaler.fit_transform(X_train)

# Create and fit LogRegression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# check the cross value score
scorerVar = make_scorer(f1_score, pos_label = 1)
lr_cv_score = cross_val_score(lr_model, X_pca, target2, cv = 10, scoring = scorerVar)
print("Cross Val Score: ", np.mean(lr_cv_score))

# predictions will output whether or not team1 wins each game
predictions = lr_model.predict(final_pca)

print(len(predictions))
print(pd.DataFrame({'team1':team1, 'team2':team2, 'predictions':predictions}))

Cross Val Score:  0.8280465082502089
36
             team1          team2  predictions
0      Connecticut        Stetson         True
1     Fla Atlantic   Northwestern        False
2     San Diego St            UAB         True
3           Auburn           Yale        False
4              BYU       Duquesne         True
5         Illinois    Morehead St         True
6       Wash State          Drake        False
7          Iowa St    S Dakota St        False
8   North Carolina         Wagner         True
9   North Carolina         Howard         True
10      Miss State    Michigan St        False
11        St Marys     Grd Canyon         True
12         Alabama  Col Charlestn         True
13         Clemson     New Mexico         True
14          Baylor        Colgate        False
15          Dayton         Nevada        False
16         Arizona    Lg Beach St         True
17         Houston       Longwood         True
18        Nebraska      Texas A&M         True
19       Wisconsin  